<a href="https://colab.research.google.com/github/Yoan-GitHub/Kobotik-Cuisine/blob/main/data_train_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import os

# Définissez le chemin vers le dossier contenant les images à renommer
path = "/content/sample_data/data"
x = 0
# Parcourez tous les fichiers dans le dossier
for filename in os.listdir(path):
    x = x + 1
    # Ignorez les dossiers et les fichiers qui ne sont pas des images
    if not filename.endswith(".jpg") and not filename.endswith(".jpeg") and not filename.endswith(".png"):
        continue
    # Vérifiez si le nom de l'image commence par "laitue_laitue"
    if filename.startswith("laitue_salade_Color"):
        new_filename = "laitue_" + filename.replace("laitue_salade_Color","",1)
    elif filename.startswith("laitue"):
        new_filename = "salade_" + filename.replace("laitue","",1)
    # Construisez les chemins complets vers le fichier actuel et le nouveau fichier
    old_path = os.path.join(path, filename)
    new_path = os.path.join(path, new_filename)
    # Renommez le fichier
    os.rename(old_path, new_path)

print(x)

100


In [40]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
import cv2

# Chemin vers le dossier d'images
image_folder = '/content/sample_data/data'  # ce dossier contient des images de laitue ( label : laitue), de laitue+pates ( label : salade )

# Génération des jeux de données d'entraînement et de test
data = []
labels = []
for image_path in os.listdir(image_folder):
    image = cv2.resize(cv2.imread(os.path.join(image_folder, image_path)), (64, 64))
    label = image_path.split("_")[0]  # On suppose que les noms des images contiennent les labels séparés par "_"
    data.append(image)
    labels.append(label)

data = np.array(data) # X : données
data = data/255

print(data.shape)
labels = np.array(labels) # y : étiquettes
print(labels.shape)
data_train, data_test, labels_train, labels_test = train_test_split(data, labels, test_size=0.2)
print(data_train.shape)
print(data_test.shape)
print(labels_train.shape)
print(labels_test.shape)

# Initialisation du modèle CNN
model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=data_train.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=2, activation='softmax'))
# one hot encode labels
le = LabelEncoder()

# Compilation du modèle
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

labels_train= np_utils.to_categorical(le.fit_transform(labels_train))
labels_test = np_utils.to_categorical(le.transform(labels_test))


print(data_train.shape)
print(labels_train.shape)

(100, 64, 64, 3)
(100,)
(80, 64, 64, 3)
(20, 64, 64, 3)
(80,)
(20,)
(80, 64, 64, 3)
(80, 1)


In [41]:
#entrainement du modèle
model.fit(data_train, labels_train, epochs=50, batch_size=32)

Epoch 1/50
3/3 [==============================] - 1s 92ms/step - loss: 0.2028 - accuracy: 1.0000
Epoch 2/50
3/3 [==============================] - 0s 95ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 3/50
3/3 [==============================] - 0s 89ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 4/50
3/3 [==============================] - 0s 87ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 5/50
3/3 [==============================] - 0s 102ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 6/50
3/3 [==============================] - 0s 135ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 7/50
3/3 [==============================] - 0s 140ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 8/50
3/3 [==============================] - 0s 144ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 9/50
3/3 [==============================] - 0s 153ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Epoch 10/50
3/3 [==============================] - 0s 153ms/step - loss: 0.0000e+00 - accu

In [42]:
# évaluation du modèle
scores = model.evaluate(data_test, labels_test, verbose=1)
print("Accuracy:", scores[1])

1/1 [==============================] - 0s 179ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Accuracy: 1.0


In [43]:
# sauvegarde du modèle
model.save('model.h5')

In [45]:
# chargement du modèle sauvegardé
import keras
loaded_model = keras.models.load_model('model.h5')

In [46]:
# prédictions sur les données de test
predictions = loaded_model.predict(data_test)
print(np.argmax(predictions[0]))

1/1 [==============================] - 0s 108ms/step
1


In [48]:
# pour afficher la catégorie prédite
print(le.inverse_transform(np.argmax(predictions[0])))

NotFittedError: ignored

In [ ]:
# one hot encode label
le = LabelEncoder()
labels_train = le.fit_transform(labels_train)
labels_test = le.transform(labels_test)
labels_train = np_utils.to_categorical(labels_train)
labels_test = np_utils.to_categorical(labels_test)

In [ ]:
# entrainement du modèle 
history = model.fit(data_train, labels_train, epochs=10, validation_data=(data_test, labels_test))

In [ ]:
# évaluation du modèle sur les données de test
scores = model.evaluate(data_test, labels_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

In [ ]:
# sauvegarde du modèle
model.save('/content/sample_data/model.h5')

In [ ]:
# récupération des données d'entrainement pour visualisation
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
# plot de l'accuracy
import matplotlib.pyplot as plt
epochs = range(len(acc))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

In [ ]:
# plot de la loss
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()